In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import numpy as np
import os

In [13]:
match_rep_df = pd.read_csv("checkpoints/match_rep_df.csv")

experiment_df = pd.read_csv("output/all-profile-metadata.csv")
experiment_df

,Vendor,Batch,Plate_Map_Name,Assay_Plate_Barcode,Modality,Images_per_well,Sites-SubSampled,Binning,Magnification,Number_of_channels,z_plane,BF_Zplanes,spinning-disc,aperture,dry-immersion,vs-brightfield,simultaneous-excitation,sites,Size_MB,Size_MB_std
0,MolDev,Scope1_MolDev_10X,JUMP-MOA_compound_platemap,Plate2_PCO_6ch_4site_10XPA,Confocal,4,NaN,1,10,6,1,NaN,no,0.45,dry,NaN,NaN,4,52.476334,0.000144
1,MolDev,Scope1_MolDev_10X,JUMP-MOA_compound_platemap,Plate3_PCO_6ch_4site_10XPA_Crest,Confocal,4,NaN,1,10,6,1,NaN,yes,0.45,dry,NaN,NaN,4,52.476514,0.000183
2,MolDev,Scope1_MolDev_10X_4siteZ,JUMP-MOA_compound_platemap,Plate3_PCO_6ch_4site_10XPA_Crestz,Confocal,4,NaN,1,10,6,3,NaN,yes,0.45,dry,NaN,NaN,4,52.476631,0.000142
3,MolDev,Scope1_MolDev_20X_4site,JUMP-MOA_compound_platemap,Plate3_PCO_6ch_4site_20XPA_Crestz,Confocal,4,NaN,1,20,6,3,NaN,yes,0.75,dry,NaN,NaN,4,52.476629,0.000114
4,MolDev,Scope1_MolDev_20X_9site,JUMP-MOA_compound_platemap,Plate2_PCO_6ch_9site_20XPA,Confocal,9,NaN,1,20,6,1,NaN,no,0.75,dry,NaN,NaN,9,52.476334,0.000153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,Yokogawa_US,4siteSubSample_Scope1_Yokogawa_US_20X_5Ch,JUMP-MOA_compound_platemap,BRO0117056_20x,Confocal,9,4.0,1,20,5,12,13,NaN,1.00,water,NaN,NaN,4,8.003585,0.000044
180,Yokogawa_US,4siteSubSample_Scope1_Yokogawa_US_20X_5Ch_12Z,JUMP-MOA_compound_platemap,BRO0117056_20xb,Confocal,9,4.0,1,20,5,12,15,NaN,1.00,water,NaN,NaN,4,8.003585,0.000044
181,Yokogawa_US,4siteSubSample_Scope1_Yokogawa_US_20X_6Ch_BRO0...,JUMP-MOA_compound_platemap,BRO0117059_20X,Confocal,9,4.0,1,20,6,12,15,NaN,1.00,water,NaN,NaN,4,7.954439,0.000583
182,Yokogawa_US,4siteSubSample_Scope1_Yokogawa_US_20X_6Ch_BRO0...,JUMP-MOA_compound_platemap,BRO01177034_20x,Confocal,9,4.0,1,20,6,12,15,NaN,1.00,water,NaN,NaN,4,8.003584,0.000014


In [128]:
# Find channel names from load_data_csvs

def find_channel_names(metadata_df, load_data_path):
    """
    Find channel names from the load_data_with_illum csv files

    Then, check the feature.csv.gz file to double-check that these column names 
    exist there too
    """
    plates = []
    df = metadata_df.copy()
    col = list()

    for root, dirs, files in os.walk(load_data_path):
        for file in files:
            if "load_data_with_illum.csv" in file:
                path = os.path.join(root, file)
                plate = os.path.split(path)[0].split("/")[-1]
                plates.append(plate)
                if plate not in metadata_df["Assay_Plate_Barcode"].values:
                    print(path, plate)
                load_data = pd.read_csv(path)
                columns = load_data.columns.tolist()
                col.append(columns)
                channels = sorted([i.replace("FileName_Orig", "") for i in columns if "FileName_Orig" in i])
                channel_string = ", ".join(channels)

                df.loc[df["Assay_Plate_Barcode"] == plate, "channel_names"] = channel_string
                if "FileName_OrigBrightField" in columns:
                    file_list = load_data["FileName_OrigBrightField"].values
                    z_pos = list(set([i[-10:-7] for i in file_list]))
                    print(plate, set(z_pos))
                    # z_pos[0] to make z_pos not an iterable
                    df.loc[df["Assay_Plate_Barcode"] == plate, "brightfield_z_plane_used"] = z_pos[0]
    # df["feature_channels_found"] = ""
    return df
    # return col

df = find_channel_names(experiment_df, "../jump-scope/load_data_csv/")

def find_channels_in_features(metadata_df, profile_parent_dir="../jump-scope/profiles"):
    df = metadata_df.copy()
    for ind, row in metadata_df.iterrows():
        for root, dirs, files in os.walk(profile_parent_dir):
            load_path = os.path.join(profile_parent_dir, row["Batch"], row["Assay_Plate_Barcode"],  row["Assay_Plate_Barcode"]+".csv.gz")
            for file in files:
                if os.path.join(root, file) == load_path:
                    load_df = pd.read_csv(load_path)
                    channels = row["channel_names"].split(", ")
                    found_chan = []
                    for chan in channels:
                        bf_df = load_df.loc[:, load_df.columns.str.contains(chan)].columns
                        if len(bf_df) > 0:
                            found_chan.append(chan)
                    found_chan = ", ".join(found_chan)
                    print(found_chan, row["Assay_Plate_Barcode"])
                    df.loc[df["Assay_Plate_Barcode"] == row["Assay_Plate_Barcode"], "feature_channels_found"] = found_chan
    return df

# df = find_channel_names(experiment_df[:2], "../jump-scope/load_data_csv/")

df = find_channels_in_features(df)

df.to_csv("test.csv")

# set(experiment_df.channel_names)
# df[~df["brightfield_z_plane"].isnull()][["Batch", "Assay_Plate_Barcode", "brightfield_z_plane"]]

# [set(i) for i in df]

# t.to_csv("test.csv")

../jump-scope/load_data_csv/2020_11_06_Scope1_MolDev/Plate3_PCO_6ch_1site_10XPA_Crestz/load_data_with_illum.csv Plate3_PCO_6ch_1site_10XPA_Crestz
../jump-scope/load_data_csv/2020_11_06_Scope1_MolDev/Plate3_PCO_6ch_1site_10XPA_Crest/load_data_with_illum.csv Plate3_PCO_6ch_1site_10XPA_Crest
../jump-scope/load_data_csv/2020_11_06_Scope1_MolDev/Plate2-PCO-6ch-1site-10XPA/load_data_with_illum.csv Plate2-PCO-6ch-1site-10XPA
BRO0117059_40x {'Z08'}
BRO0117033_20x {'Z06'}
BRO01177034_20x {'Z17'}
BRO0117059_20X {'Z08'}
BRO0117014_10x {'Z06'}
../jump-scope/load_data_csv/2020_11_16_Scope1_YokogawaUS/BRO0117014_20x/load_data_with_illum.csv BRO0117014_20x
BRO0117014_20x {'Z06'}
['Actin', 'DNA', 'ER', 'Golgi', 'Mito', 'RNA']
Index(['Cells_Correlation_Correlation_Actin_DNA',
       'Cells_Correlation_Correlation_Actin_ER',
       'Cells_Correlation_Correlation_Actin_Golgi',
       'Cells_Correlation_Correlation_Actin_Mito',
       'Cells_Correlation_Correlation_Actin_RNA',
       'Cells_Correlation_Co

In [140]:
df[(df["Vendor"] == "Yokogawa_US") & (~df["Batch"].str.contains("siteSub"))][["Vendor", "Batch", "Assay_Plate_Barcode", "brightfield_z_plane_used", "BF_Zplanes"]]

,Vendor,Batch,Assay_Plate_Barcode,brightfield_z_plane_used,BF_Zplanes
48,Yokogawa_US,Scope1_Yokogawa_US_10X,BRO0117014_10x,Z06,10
49,Yokogawa_US,Scope1_Yokogawa_US_20X_5Ch,BRO0117033_20xb,NaN,none
50,Yokogawa_US,Scope1_Yokogawa_US_20X_5Ch,BRO0117056_20x,NaN,none
51,Yokogawa_US,Scope1_Yokogawa_US_20X_5Ch_12Z,BRO0117056_20xb,NaN,none
52,Yokogawa_US,Scope1_Yokogawa_US_20X_6Ch_BRO0117033,BRO0117033_20x,Z06,11
53,Yokogawa_US,Scope1_Yokogawa_US_20X_6Ch_BRO0117059,BRO0117059_20X,Z08,15
54,Yokogawa_US,Scope1_Yokogawa_US_20X_6Ch_BRO01177034,BRO01177034_20x,Z17,13
55,Yokogawa_US,Scope1_Yokogawa_US_40X_BRO0117059,BRO0117059_40x,Z08,15


In [159]:
df.to_csv("test.csv")

In [183]:
df1 = df.copy()
df["BF_Zplanes"].replace("none", np.nan, inplace=True)

In [170]:
# So, 

df1[(df1["Vendor"] == "Yokogawa_US") & (~df1["Batch"].str.contains("siteSub"))][["Vendor", "Batch", "Assay_Plate_Barcode", "brightfield_z_plane_used", "BF_Zplanes"]]

,Vendor,Batch,Assay_Plate_Barcode,brightfield_z_plane_used,BF_Zplanes
48,Yokogawa_US,Scope1_Yokogawa_US_10X,BRO0117014_10x,Z06,10
49,Yokogawa_US,Scope1_Yokogawa_US_20X_5Ch,BRO0117033_20xb,NaN,NaN
50,Yokogawa_US,Scope1_Yokogawa_US_20X_5Ch,BRO0117056_20x,NaN,NaN
51,Yokogawa_US,Scope1_Yokogawa_US_20X_5Ch_12Z,BRO0117056_20xb,NaN,NaN
52,Yokogawa_US,Scope1_Yokogawa_US_20X_6Ch_BRO0117033,BRO0117033_20x,Z06,11
53,Yokogawa_US,Scope1_Yokogawa_US_20X_6Ch_BRO0117059,BRO0117059_20X,Z08,15
54,Yokogawa_US,Scope1_Yokogawa_US_20X_6Ch_BRO01177034,BRO01177034_20x,Z17,13
55,Yokogawa_US,Scope1_Yokogawa_US_40X_BRO0117059,BRO0117059_40x,Z08,15


In [171]:
df1.loc[df1["Batch"].str.contains("siteSub"), ["BF_Zplanes", "Assay_Plate_Barcode"]] = np.nan

In [195]:
df[(df["Vendor"] == "Yokogawa_US") & (df["Batch"].str.contains("siteSub"))][["Vendor", "Batch", "Assay_Plate_Barcode", "brightfield_z_plane_used", "BF_Zplanes"]]

,Vendor,Batch,Assay_Plate_Barcode,brightfield_z_plane_used,BF_Zplanes
99,Yokogawa_US,1siteSubSample_Scope1_Yokogawa_US_10X,BRO0117014_10x,Z06,NaN
100,Yokogawa_US,1siteSubSample_Scope1_Yokogawa_US_20X_5Ch,BRO0117033_20xb,NaN,NaN
101,Yokogawa_US,1siteSubSample_Scope1_Yokogawa_US_20X_5Ch,BRO0117056_20x,NaN,NaN
102,Yokogawa_US,1siteSubSample_Scope1_Yokogawa_US_20X_5Ch_12Z,BRO0117056_20xb,NaN,NaN
103,Yokogawa_US,1siteSubSample_Scope1_Yokogawa_US_20X_6Ch_BRO0...,BRO0117059_20X,Z08,NaN
104,Yokogawa_US,1siteSubSample_Scope1_Yokogawa_US_20X_6Ch_BRO0...,BRO01177034_20x,Z17,NaN
105,Yokogawa_US,1siteSubSample_Scope1_Yokogawa_US_40X_BRO0117059,BRO0117059_40x,Z08,NaN
148,Yokogawa_US,2siteSubSample_Scope1_Yokogawa_US_10X,BRO0117014_10x,Z06,NaN
149,Yokogawa_US,2siteSubSample_Scope1_Yokogawa_US_20X_5Ch,BRO0117033_20xb,NaN,NaN
150,Yokogawa_US,2siteSubSample_Scope1_Yokogawa_US_20X_5Ch,BRO0117056_20x,NaN,NaN


In [180]:
df.columns

Index(['Vendor', 'Batch', 'Plate_Map_Name', 'Assay_Plate_Barcode', 'Modality',
       'Images_per_well', 'Sites-SubSampled', 'Binning', 'Magnification',
       'Number_of_channels', 'z_plane', 'BF_Zplanes', 'spinning-disc',
       'aperture', 'dry-immersion', 'vs-brightfield',
       'simultaneous-excitation', 'sites', 'Size_MB', 'Size_MB_std',
       'channel_names', 'brightfield_z_plane_used', 'feature_channels_found'],
      dtype='object')